In [1]:
import soundfile
import torch
import matplotlib.pyplot as plt
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text

In [2]:

# Set test wav for attention image extraction
TEST_DATA_PATH = "./data/dev_clean"
WAV_LIST_PATH = TEST_DATA_PATH + "/wav.scp"

file_name_list = []
audio_num = 1 # selelct one of the wav in file_name_list

with open(WAV_LIST_PATH, "r") as f:
    lines = f.readlines()
    for line in lines:
        num, name = line.split(' ')
        file_name_list.append(name[:-1])

speech, rate = soundfile.read(file_name_list[audio_num])

# Prepare model
d = ModelDownloader()

ASR_MODEL_PATH = "/home_data/jmpark/espnet/tools/anaconda/envs/espnet_1.7/lib/python3.8/site-packages/espnet_model_zoo/653d10049fdc264f694f57b49849343e/exp/asr_train_asr_transformer_e18_raw_bpe_sp/54epoch.pth"
LM_MODEL_PATH = "/home_data/jmpark/espnet/tools/anaconda/envs/espnet_1.7.1/lib/python3.8/site-packages/espnet_model_zoo/653d10049fdc264f694f57b49849343e/exp/lm_train_lm_adam_bpe/17epoch.pth"

speech2text = Speech2Text(
    **d.download_and_unpack('Shinji Watanabe/librispeech_asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best'),
    # Decoding parameters are not included in the model file
    maxlenratio=0.0,
    minlenratio=0.0,
    beam_size=1,
    ctc_weight=0.4,
    lm_weight=0.6,
    penalty=0.0,
    nbest=1
)
# Add register hook for in encoder layers.
net = speech2text.asr_model

In [3]:
saved_encoder_self_attn_images = []

def attn_encoder(self, input_tensor, output_tensor):
    cols = output_tensor[1].size(1)

    for i in range(cols):
        img = output_tensor[1][0,i,:]
        saved_encoder_self_attn_images.append(img)

for name, parameter in net.named_modules():
    for i in range(18): 
        if 'encoder.encoders.'+ str(i) +'.self_attn' == name:
            print(name)
            print(parameter)
            parameter.register_forward_hook(attn_encoder)

encoder.encoders.0.self_attn
MultiHeadedAttention(
  (linear_q): Linear(in_features=512, out_features=512, bias=True)
  (linear_k): Linear(in_features=512, out_features=512, bias=True)
  (linear_v): Linear(in_features=512, out_features=512, bias=True)
  (linear_out): Linear(in_features=512, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
encoder.encoders.1.self_attn
MultiHeadedAttention(
  (linear_q): Linear(in_features=512, out_features=512, bias=True)
  (linear_k): Linear(in_features=512, out_features=512, bias=True)
  (linear_v): Linear(in_features=512, out_features=512, bias=True)
  (linear_out): Linear(in_features=512, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
encoder.encoders.2.self_attn
MultiHeadedAttention(
  (linear_q): Linear(in_features=512, out_features=512, bias=True)
  (linear_k): Linear(in_features=512, out_features=512, bias=True)
  (linear_v): Linear(in_features=512, out_features=512, bias=True)
  (linear_out): 

In [4]:
saved_decoder_self_attn_images = []

def attn_decoder(self, input_tensor, output_tensor):
    cols = output_tensor[1].size(1)

    for i in range(cols):
        img = output_tensor[1][0,i,:]
        saved_decoder_self_attn_images.append(img)

for name, parameter in net.named_modules():
    for i in range(6): 
        if 'decoder.decoders.'+ str(i) +'.self_attn' == name:
            print(name)
            print(parameter)
            parameter.register_forward_hook(attn_decoder)

decoder.decoders.0.self_attn
MultiHeadedAttention(
  (linear_q): Linear(in_features=512, out_features=512, bias=True)
  (linear_k): Linear(in_features=512, out_features=512, bias=True)
  (linear_v): Linear(in_features=512, out_features=512, bias=True)
  (linear_out): Linear(in_features=512, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
decoder.decoders.1.self_attn
MultiHeadedAttention(
  (linear_q): Linear(in_features=512, out_features=512, bias=True)
  (linear_k): Linear(in_features=512, out_features=512, bias=True)
  (linear_v): Linear(in_features=512, out_features=512, bias=True)
  (linear_out): Linear(in_features=512, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
decoder.decoders.2.self_attn
MultiHeadedAttention(
  (linear_q): Linear(in_features=512, out_features=512, bias=True)
  (linear_k): Linear(in_features=512, out_features=512, bias=True)
  (linear_v): Linear(in_features=512, out_features=512, bias=True)
  (linear_out): 

In [5]:
saved_decoder_self_attn_images = []

def attn_decoder_src(self, input_tensor, output_tensor):

    # saved_decoder_src_attn_images.append(output_tensor)

    cols = output_tensor[1].size(1)

    for i in range(cols):
        img = output_tensor[1][0,i,:]
        saved_decoder_src_attn_images.append(img)

for name, parameter in net.named_modules():
    for i in range(6): 
        if 'decoder.decoders.'+ str(i) +'.src_attn' == name:
            print(name)
            print(parameter)
            parameter.register_forward_hook(attn_decoder_src)

decoder.decoders.0.src_attn
MultiHeadedAttention(
  (linear_q): Linear(in_features=512, out_features=512, bias=True)
  (linear_k): Linear(in_features=512, out_features=512, bias=True)
  (linear_v): Linear(in_features=512, out_features=512, bias=True)
  (linear_out): Linear(in_features=512, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
decoder.decoders.1.src_attn
MultiHeadedAttention(
  (linear_q): Linear(in_features=512, out_features=512, bias=True)
  (linear_k): Linear(in_features=512, out_features=512, bias=True)
  (linear_v): Linear(in_features=512, out_features=512, bias=True)
  (linear_out): Linear(in_features=512, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
decoder.decoders.2.src_attn
MultiHeadedAttention(
  (linear_q): Linear(in_features=512, out_features=512, bias=True)
  (linear_k): Linear(in_features=512, out_features=512, bias=True)
  (linear_v): Linear(in_features=512, out_features=512, bias=True)
  (linear_out): Lin

In [11]:
saved_encoder_self_attn_images = []
saved_decoder_self_attn_images = []
saved_decoder_src_attn_images = []
out = speech2text(speech)

In [12]:
print(len(saved_encoder_self_attn_images))
print(len(saved_decoder_src_attn_images))


144
0


In [9]:
saved_decoder_src_attn_images[0]

IndexError: list index out of range

In [8]:
for name, parameter in net.named_modules():
    print(name)
    # print(parameter)


frontend
frontend.stft
frontend.frontend
frontend.logmel
specaug
specaug.time_warp
specaug.freq_mask
specaug.time_mask
normalize
encoder
encoder.embed
encoder.embed.conv
encoder.embed.conv.0
encoder.embed.conv.1
encoder.embed.conv.2
encoder.embed.conv.3
encoder.embed.out
encoder.embed.out.0
encoder.embed.out.1
encoder.embed.out.1.dropout
encoder.encoders
encoder.encoders.0
encoder.encoders.0.self_attn
encoder.encoders.0.self_attn.linear_q
encoder.encoders.0.self_attn.linear_k
encoder.encoders.0.self_attn.linear_v
encoder.encoders.0.self_attn.linear_out
encoder.encoders.0.self_attn.dropout
encoder.encoders.0.feed_forward
encoder.encoders.0.feed_forward.w_1
encoder.encoders.0.feed_forward.w_2
encoder.encoders.0.feed_forward.dropout
encoder.encoders.0.feed_forward.activation
encoder.encoders.0.norm1
encoder.encoders.0.norm2
encoder.encoders.0.dropout
encoder.encoders.1
encoder.encoders.1.self_attn
encoder.encoders.1.self_attn.linear_q
encoder.encoders.1.self_attn.linear_k
encoder.encoder